In [1]:
import openslide
import pandas as pd
import numpy as np
from tqdm import tqdm
import os
import matplotlib.pyplot as plt
from PIL import Image
import cv2
import ast
import tensorflow as tf
from tensorflow import keras
import tensorflow_addons as tfa

# Notes

The paths are based on the Kaggle repositories. \
the Navigation_v2.csv data are imported from the final-preprocessing.ipynb notebook's output.

In [2]:
df = pd.read_csv('../input/mayo-clinic-strip-ai/train.csv')

In [3]:
df['label'] = df['label'].apply(lambda x: 1 if x == 'CE' else 0)

In [4]:
df = df.set_index('image_id')

In [5]:
samples = pd.read_csv('../input/mayo-clinic-strip-ai/test.csv')

In [ ]:
final_data= pd.read_csv('../input/final-preprocessing/navigation_v2.csv')
final_data = final_data.set_index('Unnamed: 0')
final_data = final_data.astype(object)

In [7]:
def get_patch(path, tl_pixel, patch_shape):
    img = openslide.open_slide(path)
    img = np.array(img.read_region(tl_pixel, 0, patch_shape).convert("RGB"))
    return cv2.resize(img/255.0, (512, 512))

In [9]:
def data_gen(df = df , data = final_data):
    for i in range(0,650):
        n_images = 2771 - final_data.iloc[i].isnull().sum()
        if n_images <5:
            continue
        label = df.loc[final_data.index[i]]['label']
        if label == 0 :
            images_list = np.arange(n_images)
            for x in range(3):
                final_image_list = np.random.choice(images_list, size=(5), replace=False)
                img1 = get_patch('../input/mayo-clinic-strip-ai/train/'+final_data.index[i]+'.tif' , ast.literal_eval(final_data.iloc[i][final_image_list[0]]),(512,512))
                img2 = get_patch('../input/mayo-clinic-strip-ai/train/'+final_data.index[i]+'.tif' , ast.literal_eval(final_data.iloc[i][final_image_list[1]]),(512,512))
                img3 = get_patch('../input/mayo-clinic-strip-ai/train/'+final_data.index[i]+'.tif' , ast.literal_eval(final_data.iloc[i][final_image_list[2]]),(512,512))
                img4 = get_patch('../input/mayo-clinic-strip-ai/train/'+final_data.index[i]+'.tif' , ast.literal_eval(final_data.iloc[i][final_image_list[3]]),(512,512))
                img5 = get_patch('../input/mayo-clinic-strip-ai/train/'+final_data.index[i]+'.tif' , ast.literal_eval(final_data.iloc[i][final_image_list[4]]),(512,512))
                yield ({'input_71':img1,'input_72':img2,'input_73':img3,'input_74':img4,'input_75':img5},label)
        else:
            images_list = np.arange(n_images)
            for x in range(1):
                final_image_list = np.random.choice(images_list, size=(5), replace=False)
                img1 = get_patch('../input/mayo-clinic-strip-ai/train/'+final_data.index[i]+'.tif' , ast.literal_eval(final_data.iloc[i][final_image_list[0]]),(512,512))
                img2 = get_patch('../input/mayo-clinic-strip-ai/train/'+final_data.index[i]+'.tif' , ast.literal_eval(final_data.iloc[i][final_image_list[1]]),(512,512))
                img3 = get_patch('../input/mayo-clinic-strip-ai/train/'+final_data.index[i]+'.tif' , ast.literal_eval(final_data.iloc[i][final_image_list[2]]),(512,512))
                img4 = get_patch('../input/mayo-clinic-strip-ai/train/'+final_data.index[i]+'.tif' , ast.literal_eval(final_data.iloc[i][final_image_list[3]]),(512,512))
                img5 = get_patch('../input/mayo-clinic-strip-ai/train/'+final_data.index[i]+'.tif' , ast.literal_eval(final_data.iloc[i][final_image_list[4]]),(512,512))
                yield ({'input_71':img1,'input_72':img2,'input_73':img3,'input_74':img4,'input_75':img5},label)
       
    
    
def test_gen(df = df , data = final_data):
    for i in range(650,754):
        n_images = 2771 - final_data.iloc[i].isnull().sum()
        if n_images <5:
            continue
        label = df.loc[final_data.index[i]]['label']
        if label == 0 :
            images_list = np.arange(n_images)
            final_image_list = np.random.choice(images_list, size=(5), replace=False)
            img1 = get_patch('../input/mayo-clinic-strip-ai/train/'+final_data.index[i]+'.tif' , ast.literal_eval(final_data.iloc[i][final_image_list[0]]),(512,512))
            img2 = get_patch('../input/mayo-clinic-strip-ai/train/'+final_data.index[i]+'.tif' , ast.literal_eval(final_data.iloc[i][final_image_list[1]]),(512,512))
            img3 = get_patch('../input/mayo-clinic-strip-ai/train/'+final_data.index[i]+'.tif' , ast.literal_eval(final_data.iloc[i][final_image_list[2]]),(512,512))
            img4 = get_patch('../input/mayo-clinic-strip-ai/train/'+final_data.index[i]+'.tif' , ast.literal_eval(final_data.iloc[i][final_image_list[3]]),(512,512))
            img5 = get_patch('../input/mayo-clinic-strip-ai/train/'+final_data.index[i]+'.tif' , ast.literal_eval(final_data.iloc[i][final_image_list[4]]),(512,512))
            yield ({'input_71':img1,'input_72':img2,'input_73':img3,'input_74':img4,'input_75':img5},label)
        else:
            images_list = np.arange(n_images)
            final_image_list = np.random.choice(images_list, size=(5), replace=False)
            img1 = get_patch('../input/mayo-clinic-strip-ai/train/'+final_data.index[i]+'.tif' , ast.literal_eval(final_data.iloc[i][final_image_list[0]]),(512,512))
            img2 = get_patch('../input/mayo-clinic-strip-ai/train/'+final_data.index[i]+'.tif' , ast.literal_eval(final_data.iloc[i][final_image_list[1]]),(512,512))
            img3 = get_patch('../input/mayo-clinic-strip-ai/train/'+final_data.index[i]+'.tif' , ast.literal_eval(final_data.iloc[i][final_image_list[2]]),(512,512))
            img4 = get_patch('../input/mayo-clinic-strip-ai/train/'+final_data.index[i]+'.tif' , ast.literal_eval(final_data.iloc[i][final_image_list[3]]),(512,512))
            img5 = get_patch('../input/mayo-clinic-strip-ai/train/'+final_data.index[i]+'.tif' , ast.literal_eval(final_data.iloc[i][final_image_list[4]]),(512,512))
            yield ({'input_71':img1,'input_72':img2,'input_73':img3,'input_74':img4,'input_75':img5},label)

In [ ]:
dataset = tf.data.Dataset.from_generator(
     data_gen,
     ({'input_71':tf.float32,'input_72':tf.float32,'input_73':tf.float32,'input_74':tf.float32,'input_75':tf.float32}, tf.float32),
    ({'input_71':tf.TensorShape([512,512,3]),'input_72':tf.TensorShape([512,512,3]),'input_73':tf.TensorShape([512,512,3]),'input_74':tf.TensorShape([512,512,3]),'input_75':tf.TensorShape([512,512,3])}, tf.TensorShape([]))
)
testset = tf.data.Dataset.from_generator(
     test_gen,
     ({'input_71':tf.float32,'input_72':tf.float32,'input_73':tf.float32,'input_74':tf.float32,'input_75':tf.float32}, tf.float32),
    ({'input_71':tf.TensorShape([512,512,3]),'input_72':tf.TensorShape([512,512,3]),'input_73':tf.TensorShape([512,512,3]),'input_74':tf.TensorShape([512,512,3]),'input_75':tf.TensorShape([512,512,3])}, tf.TensorShape([]))
)

In [11]:

dataset = dataset.batch(4)
dataset=dataset.prefetch(1)
dataset = dataset.shuffle(buffer_size = 10, seed=3)
testset = testset.batch(4)
testset=testset.prefetch(1)

In [12]:
resnet = tf.keras.applications.EfficientNetB1(
    include_top=False,
    weights='imagenet',
    input_tensor=None,
    input_shape=(512,512,3),
    pooling=None,
)
def augmentation(A):
    A = tf.keras.layers.RandomFlip()(A)
    A =tf.keras.layers.RandomRotation(
        (-0.2, 0.2),
        fill_mode='reflect',
        interpolation='bilinear',
        seed=None,
        fill_value=0.0,
    )(A)
    A = tf.keras.layers.RandomZoom(
        (-0.1,0.1),
        (-0.1,0.1),
        fill_mode='reflect',
        interpolation='bilinear',
        seed=None,
        fill_value=0.0,
    )(A)
    return A

27033600/27018416 [==============================] - 0s 0us/step


In [14]:
inputA = keras.layers.Input(shape=(512,512,3),name='input_71')
inputB = keras.layers.Input(shape=(512,512,3),name='input_72')
inputC = keras.layers.Input(shape=(512,512,3),name='input_73')
inputD = keras.layers.Input(shape=(512,512,3),name='input_74')
inputE = keras.layers.Input(shape=(512,512,3),name='input_75')

A = augmentation(inputA)
A = resnet(A)
A = keras.models.Model(inputs=inputA, outputs=A)
B = augmentation(inputB)
B = resnet(B)
B = keras.models.Model(inputs=inputB, outputs=B)
C = augmentation(inputC)
C = resnet(C)
C = keras.models.Model(inputs=inputC, outputs=C)
D = resnet(inputD)
D = keras.models.Model(inputs=inputD, outputs=D)
E = resnet(inputE)
E = keras.models.Model(inputs=inputE, outputs=E)

X = keras.layers.Concatenate(axis=-1)([A.output, B.output, C.output, D.output, E.output])
X = keras.layers.Conv2D(128,3)(X)
X = keras.layers.AveragePooling2D()(X) 
X = keras.layers.Conv2D(32,3)(X)
X = keras.layers.AveragePooling2D()(X)
X = keras.layers.Flatten()(X)
X = keras.layers.BatchNormalization()(X)

X = keras.layers.Dense(64 ,activation = 'relu')(X)
X = keras.layers.Dense(32 ,activation = 'relu')(X)
X = keras.layers.Dense(16 ,activation = 'relu')(X)
X = keras.layers.Dense(8 ,activation = 'relu')(X)
X = keras.layers.Dense(1 ,activation = 'sigmoid')(X)
                                               
model = keras.models.Model(inputs=[A.input,B.input,C.input,D.input,E.input], outputs=X)

In [15]:
model.summary()

Model: "model_8"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_71 (InputLayer)           [(None, 512, 512, 3) 0                                            
__________________________________________________________________________________________________
input_72 (InputLayer)           [(None, 512, 512, 3) 0                                            
__________________________________________________________________________________________________
input_73 (InputLayer)           [(None, 512, 512, 3) 0                                            
__________________________________________________________________________________________________
random_flip_3 (RandomFlip)      (None, 512, 512, 3)  0           input_71[0][0]                   
____________________________________________________________________________________________

In [21]:
opt = tfa.optimizers.AdamW(
    weight_decay=1e-4,
    learning_rate = 0.00005,
    beta_1 = 0.9,
    beta_2 = 0.999,
    epsilon = 1e-07,
    name = 'AdamW',
)
loss = tf.keras.losses.BinaryCrossentropy(
    reduction=tf.keras.losses.Reduction.AUTO,
    name='binary_crossentropy'
)

In [22]:
model.compile(optimizer=opt,
              loss=loss,
                metrics=[tf.keras.metrics.Recall(), tf.keras.metrics.Precision()])

In [23]:
!mkdir model2

In [24]:
model_checkpoint_callback_LASSO = tf.keras.callbacks.ModelCheckpoint(
    filepath = './model2/model',
    monitor="val_loss",
    save_best_only=True,
    save_weights_only=True,
    mode="auto",
)

In [ ]:
H = model.fit(dataset, validation_data=testset, epochs=10, callbacks=[model_checkpoint_callback_LASSO])

In [ ]:
def get_patch1(img, tl_pixel, patch_shape):
    img = np.array(img.read_region(tl_pixel, 0, patch_shape).convert("RGB"))
    return img

final_samples= pd.DataFrame()
ids = []
for n in tqdm(range(samples.shape[0])):
    img_path = "../input/mayo-clinic-strip-ai/test/"+samples.iloc[n]['image_id']+".tif"
   #label = samples.iloc[n]['label']
    i_id = samples.iloc[n]['image_id']
    ids.append(i_id)
    img  = openslide.open_slide(img_path)
    series = pd.Series()
    n = 0
    for i in range(img.dimensions[0]//20,img.dimensions[0]-img.dimensions[0]//20,512):
        for j in range(img.dimensions[0]//20,img.dimensions[0]-img.dimensions[0]//20,512):
            if n ==20:
                break           
            im = get_patch1(img , (i,j), (512,512) )
            if im.std()>10.0:
                n=n+1
                s = pd.Series(data = [(i,j)])
                series = series.append(s,ignore_index=True)
                series = series.reset_index()
                series = series.drop("index",axis=1)
    final_samples = pd.concat([final_samples , series],axis=1)
    final_samples.columns = ids
final_samples = final_samples.T


In [ ]:
def sample_gen(df = samples , data = final_samples):
    for i in range(samples.shape[0]):
        img1 = get_patch('../input/mayo-clinic-strip-ai/test/'+df.iloc[i]['image_id']+'.tif' , (data.loc[df.iloc[i]['image_id']][0]),(512,512))
        img2 = get_patch('../input/mayo-clinic-strip-ai/test/'+df.iloc[i]['image_id']+'.tif' , (data.loc[df.iloc[i]['image_id']][1]),(512,512))
        img3 = get_patch('../input/mayo-clinic-strip-ai/test/'+df.iloc[i]['image_id']+'.tif' , (data.loc[df.iloc[i]['image_id']][2]),(512,512))
        img4 = get_patch('../input/mayo-clinic-strip-ai/test/'+df.iloc[i]['image_id']+'.tif' , (data.loc[df.iloc[i]['image_id']][3]),(512,512))
        img5 = get_patch('../input/mayo-clinic-strip-ai/test/'+df.iloc[i]['image_id']+'.tif' , (data.loc[df.iloc[i]['image_id']][4]),(512,512))
        yield ({'input_71':img1,'input_72':img2,'input_73':img3,'input_74':img4,'input_75':img5})
sampleset = tf.data.Dataset.from_generator(
     sample_gen,
     ({'input_71':tf.float32,'input_72':tf.float32,'input_73':tf.float32,'input_74':tf.float32,'input_75':tf.float32}),
    ({'input_71':tf.TensorShape([512,512,3]),'input_72':tf.TensorShape([512,512,3]),'input_73':tf.TensorShape([512,512,3]),'input_74':tf.TensorShape([512,512,3]),'input_75':tf.TensorShape([512,512,3])}))


In [ ]:
weight_file = "./model"
model.load_weights(weight_file).expect_partial()
print("Weights loaded successfully")

In [ ]:
sampleset = sampleset.batch(1)

In [ ]:
pred = model.predict(sampleset)
pred

In [ ]:
result = pd.DataFrame({'patient_id':samples['patient_id'],'CE': pred[:,0],'LAA': 1-pred[:,0]}).groupby("patient_id").mean()
submission = result[["CE", "LAA"]].round(6).reset_index()
submission

## Submission

In [ ]:
submission.to_csv('./submission.csv')